In [1]:
import os
from torch.utils.data import Dataset
from PIL import Image
import numpy as np
import torch
from torchvision import transforms

class MyTransform:
    def __init__(self):
        self.image_transform = transforms.Compose([
            transforms.Resize((256, 512)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        self.mask_transform = transforms.Compose([
            transforms.Resize((256, 512), interpolation=transforms.InterpolationMode.NEAREST)
        ])

    def __call__(self, image, mask):
        image = self.image_transform(image)
        mask = self.mask_transform(mask)
        mask = torch.tensor(np.array(mask), dtype=torch.long)  # Convert mask to tensor manually

        # Cityscapes remap specific values
        class_remap = {
            0: 0,  # Void
            1: 1,  # Road
            2: 2,  # Sidewalk
            3: 3,  # Building
            4: 4,  # Wall
            5: 5,  # Fence
            6: 6,  # Pole
            7: 7,  # Traffic light
            8: 8,  # Traffic sign
            9: 9,  # Vegetation
            10: 10,  # Terrain
            11: 11,  # Sky
            12: 12,  # Person
            13: 13,  # Rider
            14: 14,  # Car
            15: 15,  # Truck
            16: 16,  # Bus
            17: 17,  # Train
            18: 18,  # Motorcycle
            19: 18,  # Bicycle remapped to class 18 (if necessary)
            # Add other remapping as necessary
        }
        
        for old_class, new_class in class_remap.items():
            mask[mask == old_class] = new_class

        return image, mask

class CityscapesDataset(Dataset):
    def __init__(self, root_dir, split='train', transform=None):
        self.root_dir = root_dir
        self.split = split
        self.transform = transform
        self.images = self._get_images()

    def _get_images(self):
        image_dir = os.path.join(self.root_dir, 'leftImg8bit', self.split)
        images = []
        for subdir, dirs, files in os.walk(image_dir):
            for file in files:
                if file.endswith("_leftImg8bit.png"):
                    images.append(os.path.join(subdir, file))
        return images

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        img_path = self.images[index]
        base_dir = os.path.dirname(img_path)
        filename = os.path.basename(img_path)
        mask_filename = filename.replace('leftImg8bit', 'gtFine_labelIds')
        mask_path = os.path.join(base_dir.replace('leftImg8bit', 'gtFine'), mask_filename)

        image = Image.open(img_path).convert("RGB")
        mask = Image.open(mask_path).convert("L")

        # Print unique values before transformation
        mask_np = np.array(mask)
        unique_values_before = np.unique(mask_np)

        if self.transform:
            image, mask = self.transform(image, mask)

        # Print unique values after transformation
        unique_values_after = torch.unique(mask)

        return image, mask

transform = MyTransform()

train_dataset = CityscapesDataset(
    root_dir='/home/maith/Desktop/cityscapes', 
    split='train', 
    transform=transform
)

val_dataset = CityscapesDataset(
    root_dir='/home/maith/Desktop/cityscapes', 
    split='val', 
    transform=transform
)

print("Training samples:", len(train_dataset))
print("Validation samples:", len(val_dataset))

Training samples: 2975
Validation samples: 500


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models

class PyramidPoolingModule(nn.Module):
    def __init__(self, in_channels, pool_sizes):
        super(PyramidPoolingModule, self).__init__()
        self.pools = [nn.AdaptiveAvgPool2d(output_size=size) for size in pool_sizes]
        self.conv_blocks = nn.ModuleList([nn.Conv2d(in_channels, 512, 1) for _ in pool_sizes])
        self.batch_norms = nn.ModuleList([nn.BatchNorm2d(512) for _ in pool_sizes])

    def forward(self, x):
        features = [x]
        for pool, conv, bn in zip(self.pools, self.conv_blocks, self.batch_norms):
            pooled = pool(x)
            convolved = conv(pooled)
            upsampled = F.interpolate(convolved, size=x.shape[2:], mode='bilinear', align_corners=False)
            features.append(bn(upsampled))
        return torch.cat(features, dim=1)

class CascadeFeatureFusion(nn.Module):
    def __init__(self, low_channels, high_channels, out_channels, num_classes):
        super(CascadeFeatureFusion, self).__init__()
        self.conv_low = nn.Conv2d(low_channels, out_channels, 3, padding=2, dilation=2)
        self.conv_high = nn.Conv2d(high_channels, out_channels, 1)
        self.conv_low_bn = nn.BatchNorm2d(out_channels)
        self.conv_high_bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.class_conv = nn.Conv2d(out_channels, num_classes, 1)

    def forward(self, low_res_input, high_res_input):
        low_res = self.relu(self.conv_low_bn(self.conv_low(low_res_input)))
        high_res = self.relu(self.conv_high_bn(self.conv_high(high_res_input)))
        
        # Resize high_res to match low_res spatial dimensions
        high_res = F.interpolate(high_res, size=low_res.shape[2:], mode='bilinear', align_corners=False)
        
        result = low_res + high_res
        class_output = self.class_conv(result)
        return result, class_output

class ICNet(nn.Module):
    def __init__(self, num_classes):
        super(ICNet, self).__init__()
        self.backbone = models.resnet50(pretrained=True)
        self.backbone = nn.Sequential(*list(self.backbone.children())[:-2])  # Remove the last two layers (average pool and fc)
        self.ppm = PyramidPoolingModule(2048, [1, 2, 3, 6])
        self.cff1 = CascadeFeatureFusion(1024, 4096, 256, num_classes)  # Make sure channels match
        self.cff2 = CascadeFeatureFusion(512, 256, 128, num_classes)

    def forward(self, x):
        # Extract features at multiple levels
        layer1 = self.backbone[4](self.backbone[3](self.backbone[2](self.backbone[1](self.backbone[0](x)))))
        layer2 = self.backbone[5](layer1)
        layer3 = self.backbone[6](layer2)
        layer4 = self.backbone[7](layer3)

        # Apply Pyramid Pooling Module at the last layer output
        ppm_output = self.ppm(layer4)

        # Cascade Feature Fusion
        cff1_output, class_output1 = self.cff1(layer3, ppm_output)
        cff2_output, class_output2 = self.cff2(layer2, cff1_output)

        # Final classification layer
        final_output = F.interpolate(class_output2, scale_factor=4, mode='bilinear', align_corners=False)  # Scale up to input image size

        return final_output

model = ICNet(num_classes=19)  # Cityscapes has 19 classes

# Transfer the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

/home/maith/Desktop/cityscapes/enet_cityscapes/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/maith/Desktop/cityscapes/enet_cityscapes/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ICNet(
  (backbone): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0): Conv2d(64, 2

In [3]:
import torch.optim as optim
import numpy as np
from collections import Counter
from tqdm import tqdm

criterion = nn.CrossEntropyLoss()

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

from torch.utils.data import DataLoader

num_workers = 16

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=num_workers)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False, num_workers=num_workers)

In [4]:
from tqdm import tqdm

def train_one_epoch(model, criterion, optimizer, data_loader, device, epoch):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    progress_bar = tqdm(data_loader, desc=f"Epoch {epoch + 1}", leave=True)
    
    for batch_idx, (images, masks) in enumerate(progress_bar):
        images, masks = images.to(device), masks.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        
        # Squeeze the masks to remove the channel dimension
        masks = masks.squeeze(1).long()
        
        # Upsample outputs
        outputs = F.interpolate(outputs, size=masks.shape[1:], mode='bilinear', align_corners=False)
        
        loss = criterion(outputs, masks)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        
        # Calculate accuracy
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == masks).sum().item()
        total += masks.numel()
        
        # Update progress bar
        progress_bar.set_postfix({
            'loss': running_loss / (batch_idx + 1),
            'accuracy': 100 * correct / total
        })

    epoch_loss = running_loss / len(data_loader)
    epoch_accuracy = 100 * correct / total
    return epoch_loss, epoch_accuracy

In [5]:
import numpy as np

def evaluate(model, criterion, data_loader, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    all_preds = []
    all_masks = []

    with torch.no_grad():
        for images, masks in tqdm(data_loader, desc="Evaluating"):
            images, masks = images.to(device), masks.to(device)

            outputs = model(images)

            # Squeeze the masks to remove the channel dimension
            masks = masks.squeeze(1).long()

            # Upsample outputs
            outputs = F.interpolate(outputs, size=masks.shape[1:], mode='bilinear', align_corners=False)

            loss = criterion(outputs, masks)
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == masks).sum().item()
            total += masks.numel()

            all_preds.append(predicted.cpu().numpy())
            all_masks.append(masks.cpu().numpy())

    epoch_loss = running_loss / len(data_loader)
    epoch_accuracy = 100 * correct / total
    return epoch_loss, epoch_accuracy, np.concatenate(all_preds), np.concatenate(all_masks)

In [6]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

def plot_confusion_matrix(preds, masks, class_names):
    cm = confusion_matrix(masks.flatten(), preds.flatten(), labels=range(len(class_names)))
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt=".2f", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.title('Normalized Confusion Matrix')
    plt.show()

In [7]:
def plot_metrics(train_losses, val_losses, train_accuracies, val_accuracies):
    epochs = range(1, len(train_losses) + 1)

    plt.figure(figsize=(14, 6))

    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_losses, label='Train Loss')
    plt.plot(epochs, val_losses, label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_accuracies, label='Train Accuracy')
    plt.plot(epochs, val_accuracies, label='Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.legend()

    plt.tight_layout()
    plt.show()

In [8]:
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

num_epochs = 10

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=num_workers)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=num_workers)

for epoch in range(num_epochs):
    train_loss, train_accuracy = train_one_epoch(model, criterion, optimizer, train_loader, device, epoch)
    val_loss, val_accuracy, preds, masks = evaluate(model, criterion, val_loader, device)
    
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accuracies.append(train_accuracy)
    val_accuracies.append(val_accuracy)
    
    print(f'Epoch {epoch + 1}/{num_epochs} completed. Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%')

# Save the model after training
torch.save(model.state_dict(), 'trained_icnet_model.pth')

# Plot metrics
plot_metrics(train_losses, val_losses, train_accuracies, val_accuracies)

# Plot confusion matrix
class_names = [f'Class {i}' for i in range(19)]
plot_confusion_matrix(preds, masks, class_names)

Epoch 1:   0%|          | 0/186 [00:00<?, ?it/s]../aten/src/ATen/native/cuda/NLLLoss2d.cu:106: nll_loss2d_forward_kernel: block: [11,0,0], thread: [256,0,0] Assertion `t >= 0 && t < n_classes` failed.
../aten/src/ATen/native/cuda/NLLLoss2d.cu:106: nll_loss2d_forward_kernel: block: [9,0,0], thread: [256,0,0] Assertion `t >= 0 && t < n_classes` failed.
../aten/src/ATen/native/cuda/NLLLoss2d.cu:106: nll_loss2d_forward_kernel: block: [0,0,0], thread: [256,0,0] Assertion `t >= 0 && t < n_classes` failed.
../aten/src/ATen/native/cuda/NLLLoss2d.cu:106: nll_loss2d_forward_kernel: block: [9,0,0], thread: [704,0,0] Assertion `t >= 0 && t < n_classes` failed.
../aten/src/ATen/native/cuda/NLLLoss2d.cu:106: nll_loss2d_forward_kernel: block: [9,0,0], thread: [705,0,0] Assertion `t >= 0 && t < n_classes` failed.
../aten/src/ATen/native/cuda/NLLLoss2d.cu:106: nll_loss2d_forward_kernel: block: [9,0,0], thread: [706,0,0] Assertion `t >= 0 && t < n_classes` failed.
../aten/src/ATen/native/cuda/NLLLoss2d.

RuntimeError: GET was unable to find an engine to execute this computation